In [1]:
import re, os
import pandas as pd
import numpy as np
import json
from pathlib import Path
from glob import glob
from tqdm import tqdm
import pickle
import time

def writePickle(file_path, sth):
    # store list in binary file so 'wb' mode
    with open(file_path, 'wb') as fp:
        pickle.dump(sth, fp)

def loadPickle(file_path):
    # for reading also binary mode is important
    with open(file_path, 'rb') as fp:
        return pickle.load(fp)

import sys
sys.path.append("/home/privacy/")
import icd9_obj 
icd9obj = icd9_obj.ICD9_obj()
from HIPPA import HIPAA
hipaa = HIPAA()

In [3]:
# !nvidia-smi

In [4]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device, torch.cuda.device_count())

cuda 1


In [10]:
import spacy
# !python -m spacy download en_core_web_trf
spacy.prefer_gpu()
nlp = spacy.load('en_core_web_trf')

2023-05-31 05:39:09.507259: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


# 1. Generate src_phi.pickle

In [42]:
newDir = os.path.join("./re_id_history/MIMIC_reid_val_record_20230704")
contain_dict = {}
# load all the mapping phi pickle files
icd9_pickle_ls = [fn for fn in os.listdir(newDir) if '.pickle' in fn]

# Structure
- contain_dict
    - ICD9_Abbr (eg. ARF, UTI)
        - Src text
            - Re-id type
                - List of reid elements
                
contain_dict = {

    'ARF': {
        '98573.txt': {
            'date': ['January', '11/25', '11/03', '11/19',          


In [12]:
root_re_id_hist = os.path.join("./re_id_history/MIMIC_reid_val_record_20230704")
src_phi_dict = loadPickle(os.path.join(root_re_id_hist, 'src_phi.pickle'))

In [18]:
# icd9_abbr_ls = list(src_phi_dict.keys())
# for icd1_abbr in icd9_abbr_ls[2:]:
icd1_src_dict = src_phi_dict[icd1_abbr]
#     fls = list(icd1_src_dict.keys())

for fn_idx in range(1):#len(fls)):
    fn_key = fls[fn_idx]
    print(icd1_abbr, fn_key, fn_idx)

    phi_dict = icd1_src_dict[fn_key]
    1/0
    phi_keyls = phi_dict.keys()

    if 'full_name' not in src_phi_dict[icd1_abbr][fn_key].keys():
        src_phi_dict[icd1_abbr][fn_key]['full_name'] = []

    if 'first_name' in phi_keyls and 'last_name' in phi_keyls:
        content = getSrc_corpus(icd1_abbr, fn_key)
        f_name_ls = list(set(phi_dict['first_name']))
        l_name_ls = list(set(phi_dict['last_name']))

        full_name_ls = getFullName_ls(f_name_ls, l_name_ls, content)

        sent_tokenls = getTokenLsBySent(content)
        new_full_name_ls = getFullName_byTokens(f_name_ls, l_name_ls, content, sent_tokenls)

        full_name_ls.sort()
        new_full_name_ls.sort()
        if full_name_ls != new_full_name_ls:
            gpt_full_name_ls = getFullName_byChatGPT(content, sent_tokenls)
            gpt_full_name_ls = filter_gpt_name_ls(gpt_full_name_ls, f_name_ls, l_name_ls)
            full_name_ls += new_full_name_ls
            full_name_ls += gpt_full_name_ls
            full_name_ls = list(set(full_name_ls))
            full_name_ls.sort()
        src_phi_dict[icd1_abbr][fn_key]['full_name'] += full_name_ls

    # switch_full_name_phone
    if 'full_name_phone' in phi_keyls:

        if 'phone_num' not in src_phi_dict[icd1_abbr][fn_key].keys():
            src_phi_dict[icd1_abbr][fn_key]['phone_num'] = []
        for onepair in phi_dict['full_name_phone']:
#                 NOT IN THE CORRECT ORDER
            tmp_fullname, tmp_phone = re.findall(r'([\w ]*) (\d.*)', onepair)[0]
            src_phi_dict[icd1_abbr][fn_key]['full_name'].append(tmp_fullname)
            src_phi_dict[icd1_abbr][fn_key]['phone_num'].append(tmp_phone)
        src_phi_dict[icd1_abbr][fn_key].pop('full_name_phone')

ARF 98573.txt 0


ZeroDivisionError: division by zero

---
# DEV

In [ ]:
def getFullName_byChatGPT(corpus, sent_tokenls):
    def getResponse(question):
        completion = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages=[
            {"role": "user", "content": question}
          ]
        )
        return completion
    
    def partial_getResponse(token_limit = 2000):
        chunk_token_num = 0
        sent_ls = []
        name_ls = []
        for tokenls, sent  in sent_tokenls:
            if chunk_token_num <= token_limit:
                chunk_token_num += len(tokenls)
                sent_ls.append(sent)
            else:
                part_content = '\n'.join(sent_ls)
                completion = getResponse(f'{prompt_instruction}\n{part_content}')
                name_ls += completion.choices[0].message['content'].split('\n')
                # reset the parameters
                chunk_token_num = len(tokenls)
                sent_ls = [sent]
        return name_ls
    
    name_ls = []
    prompt_instruction = 'Please list all the full name in the form of "last name, first name" from the following content:'
    txt_input = f'{prompt_instruction}\n{corpus}'
    try:
        completion = getResponse(txt_input)
        name_ls += completion.choices[0].message['content'].split('\n')
    except:
        try:
            sent_ls = partial_getResponse(token_limit = 2000)
        except:
            sent_ls = partial_getResponse(token_limit = 1500)
    
    ls = []
    for name in name_ls:
        if ',' in name:
            tmp_split = name.split(',')
            if len(tmp_split) != 2: continue
            ln, fn = tmp_split
            if '.' in fn:
                fn = fn.strip().split(' ')[0].strip()
            if len(fn) > 0:
                ls.append(f'{ln.strip()}, {fn}')
    
    return list(set(ls))

# gpt_full_name_ls = getFullName_byChatGPT(content, sent_tokenls)

In [ ]:
# filter name in the src list and check the structure
def filter_gpt_name_ls(gpt_name_ls, src_f_name_ls, src_l_name_ls):
    tmp_ls = []
    for gpt_name in gpt_name_ls:
        result = re.findall(r'([A-Za-z-]*), +([A-Za-z-]*)', gpt_name)
        if len(result) > 0 and len(result[0]) == 2:
            l_n, f_n = result[0]
            if f_n in src_f_name_ls and l_n in src_l_name_ls:
                tmp_ls.append('{}, {}'.format(l_n, f_n))
    return tmp_ls

In [ ]:
def getTokenLsBySent(corpus):
    corpus_ls = corpus.split('\n\n')
    sent_token_ls = []
    for paragraph in corpus_ls:
        doc = nlp(paragraph.strip())
        for sent in doc.sents:
            sent_token_ls.append(([token.text for token in sent], sent.text) )
    return sent_token_ls

def getSrc_corpus(icd9abbr, fn):
    with open(os.path.join(src_MIMIC_dir, dir_dict[icd9abbr], fn), 'r') as fr:
        content = fr.read()
    return content

In [ ]:
def isIn(name_ls, one_sentence):# token_ls):
    capture_ls = []
    for name in name_ls:
        if name in one_sentence:
            capture_ls.append(name)
    return capture_ls

def getFullName_byTokens(first_name_ls, last_name_ls, sent_tokenls, isDebug=0):
    ls = []
    for tokenls, sent in sent_tokenls:
        got_f_name_ls = isIn(first_name_ls, sent)
        got_l_name_ls = isIn(last_name_ls, sent)
        if len(got_f_name_ls) > 0 and len(got_l_name_ls) > 0:
            # token level: double check the words is contained
            if any( '-' in n for n in got_f_name_ls) or any( '-' in n for n in got_l_name_ls):
                pass
            else:
                got_f_name_ls = isIn(got_f_name_ls, tokenls) 
                got_l_name_ls = isIn(got_l_name_ls, tokenls)
            if isDebug:
                print('getFullName_byTokens')
                print(tokenls)
                print(repr(sent))
                print(got_f_name_ls, got_l_name_ls)
                print('-'*5)
            tmp_full_name_ls = getFullName_ls(got_f_name_ls, got_l_name_ls,
                                              sent.replace('\n', ' ').replace('  ', ' '))
            if isDebug: print(f'FULL NAME -->', tmp_full_name_ls)
            if len(tmp_full_name_ls) > 0: ls += tmp_full_name_ls

    ls.sort()
    return ls

# getFullName_byTokens(first_name_ls, last_name_ls, content, isDebug=1)


In [ ]:
def getFullName_ls(f_name_ls, l_name_ls, corpus, threshold = 3, isDebug=0):
    f_name_lc_ls = [(corpus.find(name), len(name)) for name in f_name_ls]
    l_name_lc_ls = [(corpus.find(name), len(name)) for name in l_name_ls]
    
    isGotcha = 0
    full_name_ls = []
    for lc_f, len_f in f_name_lc_ls:
        for lc_l, len_l in l_name_lc_ls:
            inRange = abs(lc_f+len_f - lc_l) <= threshold or abs(lc_l + len_l - lc_f) <= threshold
#             print(inRange, abs(lc_f+len_f - lc_l), abs(lc_l + len_l - lc_f))
            if inRange:
                if lc_f < lc_l:
                    full_name = corpus[lc_f: lc_l+len_l]
                else:
                    full_name = corpus[lc_l: lc_f+len_f]
                
                full_name = full_name.strip()
                if ',' in full_name:
                    f_l_ls = full_name.split(',')
                    if len(f_l_ls) != 2: continue
                    tmp_l, tmp_f = f_l_ls
                    full_name = f'{tmp_l.strip()}, {tmp_f.strip()}'
                    
                    
                elif ' ' in full_name:
                    f_l_ls = full_name.strip().split(' ')
                    if len(f_l_ls) != 2: continue
                    tmp_f, tmp_l = f_l_ls
                    full_name = f'{tmp_l}, {tmp_f}'
                    
                    
                filter_result = [f for f in re.findall(r'((?!\w).)', full_name) \
                                 if f not in [',', ' ', '-']]
                if len(filter_result) > 0:
                    print('-'*5)
                    print('filter_result')
                    print('first_name_ls = ', f_name_ls)
                    print('last_name_ls = ', l_name_ls)
                else:
                    full_name_ls.append(full_name.strip())
                    isGotcha = 1
    return full_name_ls
# content = '- fax results to: Name: Kent ,Monty R. Address: 2300 Deer Path Circle #207 Hurstbourne Acres KY 40220 , Glen Burnie ,56059'
# first_name_ls = ['Monty']
# last_name_ls = ['Kent']
# getFullName_ls(first_name_ls, last_name_ls, content)